In [4]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

In [5]:
def audio_feature_analysis(audio_file):
    # Load audio file
    y, sr = librosa.load(audio_file, sr=None)

    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

    # Extract F0 (fundamental frequency)
    f0, voiced_flag, voiced_probs = librosa.pyin(y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))

    # Extract onsets
    onset_frames = librosa.onset.onset_detect(y=y, sr=sr)

    # Extract beat frames
    beat_frames = librosa.beat.beat_track(y, sr=sr)

    # Plot MFCCs
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(librosa.power_to_db(mfccs, ref=np.max), y_axis='mel', x_axis='time')
    plt.title('MFCCs')
    plt.colorbar()
    plt.show()

    # Plot F0
    plt.figure(figsize=(10, 4))
    times = librosa.times_like(f0)
    plt.plot(times, f0, label='F0 (fundamental frequency)', color='b')
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.title('Fundamental Frequency (F0)')
    plt.legend()
    plt.show()

    # Plot onsets
    plt.figure(figsize=(10, 4))
    onset_times = librosa.times_like(onset_frames, sr=sr)
    plt.plot(onset_times, onset_frames, label='Onsets', color='r', marker='o', linestyle='None')
    plt.xlabel('Time (s)')
    plt.ylabel('Onset Frames')
    plt.title('Onsets')
    plt.legend()
    plt.show()

    # Plot beat tracking
    plt.figure(figsize=(10, 4))
    beat_times = librosa.times_like(beat_frames, sr=sr)
    plt.plot(beat_times, beat_frames, label='Beats', color='g', marker='o', linestyle='None')
    plt.xlabel('Time (s)')
    plt.ylabel('Beat Frames')
    plt.title('Beat Tracking')
    plt.legend()
    plt.show()

In [6]:
audio_file_path = '/Users/gcportable/Documents/GitHub/gawayneLessons/files/Zaini - Wherever you would call me - TRIM.wav'
audio_feature_analysis(audio_file_path)

TypeError: beat_track() takes 0 positional arguments but 1 positional argument (and 1 keyword-only argument) were given

In [7]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

def feature_analysis(audio_file):
    # Load audio file
    y, sr = librosa.load(audio_file)

    # MFCC analysis
    hop_length = 512
    mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=256)

    plt.figure(figsize=(12, 4))
    librosa.display.specshow(mfcc, x_axis='time')
    plt.ylabel('MFCCs (1:256)')
    plt.colorbar(format='%+2.0f dB')
    plt.show()

    plt.figure(figsize=(12, 4))
    librosa.display.specshow(mfcc[1:12, :], x_axis='time')
    plt.ylabel('MFCCs (2:13)')
    plt.colorbar(format='%+2.0f dB')
    plt.show()

    # F0 analysis
    audio_vals = analyze_audio(audio_file)
    plot_audio_vals(audio_vals, audio_file, 'F0 Analysis', 'f0')

    # Onsets analysis
    get_onsets_wrapper(audio_file)

    # Beat tracking
    beat_tracking(audio_file)

def analyze_audio(audio_path):
    audio_vals = {}
    y, sr = librosa.load(audio_path)
    audio_vals['specCent'] = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=2048, hop_length=512, freq=None)
    audio_vals['specBand'] = librosa.feature.spectral_bandwidth(y=y, sr=sr, n_fft=2048, hop_length=512, freq=None)
    audio_vals['specContrast'] = librosa.feature.spectral_contrast(y=y, sr=sr, n_fft=2048, hop_length=512, freq=None)
    audio_vals['specFlatness'] = librosa.feature.spectral_flatness(y=y)

    audio_vals['f0'], _, _ = librosa.pyin(y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))
    audio_vals['f0mean'] = np.nanmean(audio_vals['f0'])

    return audio_vals

def plot_audio_vals(audio_vals, audio_path, plot_title, data_name):
    y, sr = librosa.load(audio_path)
    times = librosa.times_like(audio_vals[data_name])
    D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
    fig, ax = plt.subplots()
    img = librosa.display.specshow(D, x_axis='time', y_axis='log', ax=ax)
    ax.set(title=plot_title)
    fig.colorbar(img, ax=ax, format="%+2.f dB")
    ax.plot(times, audio_vals[data_name], label=data_name, color='cyan', linewidth=3)
    ax.legend(loc='upper right')
    plt.show()

def get_onsets_wrapper(filename):
    sig, sr = librosa.load(filename, mono=True, sr=None)
    onset_frames = get_onsets_env(sig, sr)
    print('Estimated onsets:')
    print(librosa.frames_to_time(onset_frames, sr=sr))

def get_onsets_env(sig, sr=44100):
    o_env = librosa.onset.onset_strength(sig, sr=sr)
    times = librosa.frames_to_time(np.arange(len(o_env)), sr=sr)
    onset_frames = librosa.onset.onset_detect(onset_envelope=o_env, sr=sr)

    D = np.abs(librosa.stft(sig, n_fft=4096))
    plt.figure()
    ax1 = plt.subplot(2, 1, 1)
    librosa.display.specshow(librosa.amplitude_to_db(D, ref=np.max), hop_length=512, x_axis='time', y_axis='log')
    plt.title('Estimated from Onset Envelope')
    plt.subplot(2, 1, 2, sharex=ax1)
    plt.plot(times, o_env, label='Onset strength')
    plt.vlines(times[onset_frames], 0, o_env.max(), color='r', alpha=0.9, linestyle='--', label='Onsets')
    plt.axis('tight')
    plt.legend(frameon=True, framealpha=0.75)

    return onset_frames

def beat_tracking(filename):
    y, sr = librosa.load(filename)
    S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
    log_S = librosa.power_to_db(S, ref=np.max)
    mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)
    delta_mfcc = librosa.feature.delta(mfcc)
    delta2_mfcc = librosa.feature.delta(mfcc, order=2)
    M = np.vstack([mfcc, delta_mfcc, delta2_mfcc])

    y_harmonic, y_percussive = librosa.effects.hpss(y)
    C = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr)

    tempo, beats = librosa.beat.beat_track(y=y_percussive, sr=sr)

    plt.figure(figsize=(12, 4))
    librosa.display.specshow(M)
    plt.title('MFCC-$\Delta$-$\Delta^2$')
    plt.yticks(np.arange(0, M.shape[0], 13), ['MFCC', '$\Delta$', '$\Delta^2$'])
    plt.colorbar()
    plt.show()

    plt.figure(figsize=(12, 4))
    librosa.display.specshow(librosa.util.sync(C, beats, aggregate=np.median), y_axis='chroma')
    plt.title('Beat-synchronous Chroma (median aggregation)')
    plt.colorbar()
    plt.show()

# Example Usage:
audio_file_path = 'your_audio_file_path.wav'
feature_analysis(audio_file_path)


/var/folders/x_/9x4q2pmj4rlctmx19n0hp1h00000gn/T/ipykernel_98879/501802382.py:8: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file)
/Users/gcportable/anaconda3/lib/python3.11/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: 'your_audio_file_path.wav'